# Planetary Nebula Production <a class="tocSkip">
    
This notebook is used to test and showcase the results of my first project. I use spectroscopic data from the [Multi Unit Spectroscopic Explorer](https://www.eso.org/sci/facilities/develop/instruments/muse.html) (MUSE) that has been observed as part of the [PHANGS](https://sites.google.com/view/phangs/home) collaboration.
    
I will use a set of line maps of emission lines to identify Planetary Nebula in the data an measure their brightness. This can then be used to fit an empiric relation and hence measure the distance to the galaxy.
    
This notebook is used for developement. Final code is moved to the `pymuse` packge in the `src` folder. Any production scripts reside in the `scripts` folder.

## Preparation
 
### Load Basic Packages
    
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

# some basic packages
import os                 # filesystem related stuff
import json
from pathlib import Path  # use instead of os.path and glob
import sys                # mostly replaced by pathlib

import errno      # more detailed error messages
import warnings   # handles warnings
import logging    # use logging instead of print

from collections import OrderedDict  

# packages for scientific computing
import numpy as np
import scipy as sp

# packages for creating plots and figures
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# special functions for astronomy 
from astropy.table import Table  # useful datastructure
from astropy.table import vstack # combine multiple tables

from astropy.io import fits      # open fits files
from astropy.io import ascii     # handle normal files

from astropy.wcs import WCS               # handle coordinates
from astropy.coordinates import SkyCoord  # convert pixel to sky coordinates
from astropy.visualization import simple_norm

from astropy.stats import sigma_clipped_stats  # calcualte statistics of images

import astropy.units as u        # handle units

tab10 = ['#e15759','#4e79a7','#f28e2b','#76b7b2','#59a14e','#edc949','#b07aa2','#ff9da7','#9c755f','#bab0ac']    
plt.style.use('TeX.mplstyle')

we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    #format='(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

### Read in data

this uses the `ReadLineMaps` class from the `pymuse.io` module. To use it, we first need to specify the path to the data folder

IC5332, NGC1087, NGC1365, NGC1512, NGC1566, NGC1672, NGC2835, NGC3351, NGC3627, NGC4254, NGC4535, NGC5068, NGC628

In [ ]:
from pymuse.io import ReadLineMaps

name = 'IC5332'

# first we need to specify the path to the raw data
data_raw = Path('d:\downloads\MUSEDAP')
basedir = Path('..')

# list all files in the specified directory
galaxies = [x.name for x in data_raw.iterdir() if x.is_dir()]
#print(', '.join(map(str,galaxies)))

# read in the data we will be working with and print some information
galaxy = ReadLineMaps(data_raw / name)

with fits.open(data_raw / name / f'{name}_starmask.fits') as hdul:
    star_mask = hdul[0].data
    
mask = np.zeros(galaxy.shape,dtype=bool)
mask |= star_mask.astype(bool)

In [ ]:
# look at the masks

#mask[300:650,500:900]=True # NGC1365
#mask[600:900,250:450]=True #NGC3627

img = galaxy.OIII5006_DAP.copy()

img[mask] = np.nan

fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot()

norm = simple_norm(galaxy.OIII5006_DAP,clip=False,max_percent=95)
ax.imshow(img,norm=norm,origin='lower')

## Source Detection

In [ ]:
from photutils import DAOStarFinder            # DAOFIND routine to detect sources
from photutils import IRAFStarFinder           # IRAF starfind routine to detect star

from pymuse.detection import detect_unresolved_sources

In [ ]:
with open(basedir / 'data' / 'interim' / 'parameters.json') as json_file:
    parameters = json.load(json_file)
    
setattr(galaxy,'binsize',parameters[galaxy.name]['binsize'])
setattr(galaxy,'mu',parameters[galaxy.name]['mu'])
setattr(galaxy,'alpha',parameters[galaxy.name]['power_index'])
setattr(galaxy,'completeness_limit',parameters[galaxy.name]['completeness_limit'])
setattr(galaxy,'roundness',parameters[galaxy.name]['roundness'])
setattr(galaxy,'sharplo',parameters[galaxy.name]['sharplo'])
setattr(galaxy,'sharphi',parameters[galaxy.name]['sharphi'])
setattr(galaxy,'threshold',parameters[galaxy.name]['threshold'])
setattr(galaxy,'zoomin',parameters[galaxy.name]['zoomin'])


threshold = galaxy.threshold
oversize  = 1.

roundness = galaxy.roundness
sharplo   = galaxy.sharplo
sharphi   = galaxy.sharphi

print(f'threshold: {threshold}\nround:     {roundness}\nsharplo:   {sharplo}\nsharphi:   {sharphi}')

sources = detect_unresolved_sources(galaxy,
                                    'OIII5006',
                                    StarFinder=DAOStarFinder,
                                    threshold=threshold,
                                    exclude_region=mask,
                                    oversize=oversize,
                                    roundlo=-roundness,
                                    roundhi=roundness,
                                    sharplo=sharplo,
                                    sharphi=sharphi,
                                    exclude_border=True,
                                    save=False)

## Completeness limit

In [ ]:
from pymuse.detection import completeness_limit

In [ ]:
print(f'for {galaxy.name}')
mock_sources = completeness_limit(
                   galaxy,
                   'OIII5006',
                   DAOStarFinder,
                   threshold=threshold,
                   oversize=oversize,
                   iterations=5,
                   stars_per_mag=50,
                   roundlo=-roundness,
                   roundhi=roundness,
                   sharplo=sharplo,
                   sharphi=sharphi,
                                 )

## Flux measurement

In the previous step we detected potential PN candidates by their [OIII] emission. This means we know their position but lack exact flux measurments. In this section we measure the flux of the identified objects in different emission lines that are used in later steps. 

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)
from astropy.coordinates import SkyCoord

from extinction import ccm89     # calculate extinction Cardelli et al. (1989)

a_v = 0.2
r_v = 3.1

from pymuse.photometry import measure_flux

In [ ]:
flux = measure_flux(galaxy,sources,galaxy.alpha,aperture_size=2,background='local')

# calculate astronomical coordinates for comparison
flux['SkyCoord'] = SkyCoord.from_pixel(flux['x'],flux['y'],galaxy.wcs)

# calculate magnitudes from measured fluxes
flux['mOIII'] = -2.5*np.log10(flux['OIII5006']*1e-20) - 13.74
flux['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

# correct for milky way extinction
extinction = ccm89(wave=np.array([5007.]),a_v=a_v,r_v=r_v,unit='aa')[0]
flux['mOIII'] -= extinction

## Emission line diagnostics    

In [ ]:
#galaxy.completeness_limit = 28.
SNR = True

from pymuse.analyse import emission_line_diagnostics
print(f'mu={galaxy.mu:.2f}, cl={galaxy.completeness_limit}')
print(f'emission line diagnostics for {galaxy.name}')
tbl = emission_line_diagnostics(flux,galaxy.mu,galaxy.completeness_limit,SNR=SNR)

if False:
    filename = basedir / 'data' / 'catalogues' / f'pn_candidates_{galaxy.name}.txt'
    with open(filename,'w',newline='\n') as f:
        tbl['RaDec'] = tbl['SkyCoord'].to_string(style='hmsdms',precision=2)
        for col in tbl.colnames:
            if col not in ['id','RaDec','type']:
                tbl[col].info.format = '%.3f' 
        ascii.write(tbl[['id','type','x','y','RaDec','OIII5006','OIII5006_err','mOIII','dmOIII',
                         'HA6562','HA6562_err','HA6562_detection','NII6583','NII6583_err','NII6583_detection',
                         'SII6716','SII6716_err','SII6716_detection']][tbl['type']!='NaN'],
                    f,format='fixed_width',delimiter='\t',overwrite=True)
    print('data saved to ' + str(filename))   

### Visualize the result of the classification

In [ ]:
from pymuse.plot.pnlf import plot_emission_line_ratio
filename = basedir / 'reports' / f'{galaxy.name}_emission_line'
plot_emission_line_ratio(tbl,galaxy.mu,filename=filename)

In [ ]:
from pymuse.plot.classification import classification_map
parameters[galaxy.name]['zoomin'] = [100,500,400,800]
filename = basedir / 'reports' / f'{galaxy.name}_detections_classification.pdf'
classification_map(galaxy,parameters,tbl,filename)

## Planetary nebula luminosity function

In [ ]:
from pymuse.analyse import MaximumLikelihood1D, PNLF, pnlf

#galaxy.completeness_limit = 28.5

print(f'completeness limit = {galaxy.completeness_limit}')
criteria = (tbl['type']=='PN')# | (tbl['type']=='SNR')
data = tbl[np.where(criteria & (tbl['mOIII']<galaxy.completeness_limit))]['mOIII']
err  = tbl[np.where(criteria & (tbl['mOIII']<galaxy.completeness_limit))]['dmOIII']
#data = data[data>25.5]

fitter = MaximumLikelihood1D(pnlf,
                             data,
                             #err=err,
                             mhigh=galaxy.completeness_limit)

# a good guess would be mu_guess = min(data)-Mmax
print(f'{galaxy.name}: literature {galaxy.mu:.3f}')
galaxy.mu = fitter([24])
mu,dm_p,dm_m = fitter.plot()
dm_p, dm_m = np.sqrt(np.mean(err)**2+np.array([dm_p,dm_m])**2)

### Plot the fit

to plot the fit we need to bin the data

In [ ]:
from pymuse.plot.pnlf import plot_pnlf
binsize = 0.5#galaxy.binsize
print(f'plotting result for {galaxy.name}')
filename = basedir / 'reports' / f'{galaxy.name}_PNLF'
plot_pnlf(tbl[criteria]['mOIII'],mu,galaxy.completeness_limit,binsize=binsize,mhigh=32,filename=filename,color=tab10[0])

### Compare to literature

In [ ]:
from pymuse.plot.pnlf import compare_distances

filename = basedir / 'reports' / f'{name}_distances'
compare_distances(galaxy.name,mu,dm_p,dm_m,filename)

### Look at brightest objects

In [ ]:
from pymuse.plot.plot import single_cutout
k = 10
tmp = tbl[criteria].copy()

if 'SkyCoord' in tmp.colnames:
    del tmp['SkyCoord']

tmp.sort('mOIII')
print(tmp[:k][['id','mOIII','x','y','type']])

In [ ]:
idx = 230
row = tmp[tmp['id']==idx]
rgb = single_cutout(galaxy,row['x'],row['y'],size=16)

## Plot all objects in one figure

In [ ]:
with open(basedir / 'data' / 'interim' / 'parameters.json') as json_file:
    parameters = json.load(json_file)

### Measure distance to all objects

In [ ]:
d_dic = {}
for name in galaxies:
    
    completeness = parameters[name]['completeness_limit']
    
    filename = basedir / 'data' / 'catalogues' / f'pn_candidates_{name}.txt'
    tbl = ascii.read(str(filename),format='fixed_width',delimiter='\t')
    data = tbl[(tbl['type']=='PN') & (tbl['mOIII']<completeness)]['mOIII']
    err  = tbl[(tbl['type']=='PN') & (tbl['mOIII']<completeness)]['dmOIII']
    #data = data[data>26]
    
    if 'cut' in parameters[name]:
        print(f'removing points brighter than {parameters[name]["cut"]}')
        data = data[data>parameters[name]['cut']]
    
    fitter = MaximumLikelihood1D(pnlf,
                               data[data<completeness],
                               mhigh=completeness)
    
    if 'range' in parameters[name]:
        r = parameters[name]['range']
    else:
        r = [28,32]
    
    # a good guess would be mu_guess = min(data)-Mmax
    mu = fitter([24])
    d_dic[name] = fitter.plot(r)
    

In [ ]:
from pymuse.auxiliary import Distance

for k,v in d_dic.items():
    mu, mp, mm = v
    mu_pc = Distance(mu,'mu').to_parsec()/1e6
    mp_pc = 0.2*np.log(10) * mu_pc * mp
    mm_pc = 0.2*np.log(10) * mu_pc * mm
    print(f'{k} & $\\uncertainty{{{mu:.3f}}}{{{mp:.3f}}}{{{mm:.3f}}}$ & $\\uncertainty{{{mu_pc:.3f}}}{{{mp_pc:.3f}}}{{{mm_pc:.3f}}}$ \\\\')

### Plot PNLF

In [ ]:
from pymuse.analyse import MaximumLikelihood, PNLF, pnlf

Mmax = -4.47
names = galaxies
#names = ['IC5332', 'NGC1087', 'NGC1365', 'NGC1512', 'NGC1672', 'NGC2835', 'NGC3351', 'NGC3627', 'NGC4254', 'NGC4535', 'NGC5068', 'NGC628']
color = 'tab:red'

fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(6.974,6.974/3*4))

for i, ax in enumerate(fig.axes):
    
    if i < len(names):
        name = names[i]
    else:
        break
        
    if 'binsize' in parameters[name]:
        binsize=parameters[name]['binsize']
    else:
        binsize=0.4
    
    completeness = parameters[name]['completeness_limit']
    
    filename = basedir / 'data' / 'catalogues' / f'pn_candidates_{name}.txt'
    tbl = ascii.read(str(filename),format='fixed_width',delimiter='\t')
    data = tbl[(tbl['type']=='PN')]['mOIII']
    err  = tbl[(tbl['type']=='PN')]['dmOIII']
    
    if 'cut' in parameters[name]:
        print(f'removing points brighter than {parameters[name]["cut"]}')
        data = data[data>parameters[name]['cut']]

    fitter = MaximumLikelihood(pnlf,
                               data[data<completeness],
                               mhigh=completeness)
    
    # a good guess would be mu_guess = min(data)-Mmax
    mu = fitter([24])
    
    #mu = parameters[name]['mu']
    
    mlow = Mmax+mu
    mhigh = 32

    N = len(data[data<completeness])
    
    hist, bins  = np.histogram(data,np.arange(mlow,mhigh,binsize),normed=False)
    err = np.sqrt(hist)
    # midpoint of the bins is used as position for the plots
    m = (bins[1:]+bins[:-1]) / 2
    
    # for the fit line we use a smaller binsize
    binsize_fine = 0.05
    bins_fine = np.arange(mlow,mhigh,binsize_fine)
    m_fine = (bins_fine[1:]+bins_fine[:-1]) /2

    # scatter plot
    ax.errorbar(m[m<completeness],hist[m<completeness],yerr=err[m<completeness],
                 marker='o',ms=6,mec=color,mfc=color,ls='none',ecolor=color)
    ax.errorbar(m[m>=completeness],hist[m>=completeness],yerr=err[m>completeness],
                 marker='o',ms=6,mec=color,mfc='white',ls='none',ecolor=color)
    ax.plot(m_fine,binsize/binsize_fine*N*PNLF(bins_fine,mu=mu,mhigh=completeness),c=color,ls='dotted')
    #ax.axvline(completeness,c='black',lw=0.2)
    #ax.axvline(mu+Mmax,c='black',lw=0.2)

    # adjust plot
    ax.set_yscale('log')
    ax.set_xlim([1.1*mlow-0.1*mhigh,mhigh])
    ax.set_ylim([0.8,1.5*np.max(hist)])
    ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    ax.set_ylabel(r'$N$')
    
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.2g}'.format(y)))
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.25))
    ax.set_title(name)
    
plt.tight_layout()
filename = basedir / 'reports' / f'all_objects_PNLF'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.show()

### Plot cumulative PNLF

In [ ]:
from pymuse.analyse import MaximumLikelihood1D, PNLF, pnlf

binsize=0.2
Mmax = -4.47
color = 'tab:red'

fig, axes = plt.subplots(nrows=4,ncols=3,figsize=(6.974,6.974/3*4))

for i, ax in enumerate(fig.axes):
    
    if i < len(galaxies):
        name = galaxies[i]
    else:
        break
        
    completeness = parameters[name]['completeness_limit']
    
    filename = basedir / 'data' / 'catalogues' / f'pn_candidates_{name}.txt'
    tbl = ascii.read(str(filename),format='fixed_width',delimiter='\t')
    data = tbl[(tbl['type']=='PN')]['mOIII']
    err  = tbl[(tbl['type']=='PN')]['dmOIII']
    #data = data[data>26]

    if 'cut' in parameters[name]:
        print(f'removing points brighter than {parameters[name]["cut"]}')
        data = data[data>parameters[name]['cut']]
    
    fitter = MaximumLikelihood(pnlf,
                               data[data<completeness],
                               mhigh=completeness)
    
    # a good guess would be mu_guess = min(data)-Mmax
    mu = fitter([24])[0]
    
    mlow = Mmax+mu
    mhigh = 32

    N = len(data[data<completeness])
    
    hist, bins  = np.histogram(data,np.arange(mlow,mhigh,binsize),normed=False)
    err = np.sqrt(hist)
    # midpoint of the bins is used as position for the plots
    m = (bins[1:]+bins[:-1]) / 2
    
    # for the fit line we use a smaller binsize
    binsize_fine = 0.05
    bins_fine = np.arange(mlow,mhigh,binsize_fine)
    m_fine = (bins_fine[1:]+bins_fine[:-1]) /2

    # scatter plot
    ax.plot(m[m<completeness],np.cumsum(hist[m<completeness]),ls='none',mfc=color,mec=color,ms=4,marker='o')
    ax.plot(m,N*np.cumsum(PNLF(bins,mu=mu,mhigh=completeness)),ls='dotted',color=color)

    
    # adjust plot    
    ax.set_xlim([mlow,completeness])
    ax.set_ylim([-0.1*N,1.1*N])
    ax.set_xlabel(r'$m_{[\mathrm{OIII}]}$ / mag')
    ax.set_ylabel(r'Cumulative N')
    
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1.0))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.25))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.16g}'.format(y)))
    ax.set_title(name)
    
plt.tight_layout()
filename = basedir / 'reports' / f'all_objects_PNLF_cum'
plt.savefig(filename.with_suffix('.pdf'),bbox_inches='tight')
plt.show()

## Distance in parsec

the measured distances are in the form of the distance modulus $\mu = m-M$ which is the difference between apparent and absolute magnitude. By defintion of the absolte magnitude, we can convert this number into a distance in pc
$$
d = 10^{\frac{\mu}{5}+1} = 10 \cdot \exp\left( \ln 10 \frac{\mu}{5} \right) \\
\delta d = \frac{\ln 10}{5} 10 \exp\left( \ln 10 \frac{\mu}{5} \right) \delta \mu = 0.2 \ln 10 \; d \; \delta \mu
$$

In [ ]:
def distance_modulus_to_parsec(mu,mu_err=np.array([])):
    
    d = 10 * np.exp(np.log(10)*mu/5)
    if len(mu_err) > 0:
        d_err = 0.2 * np.log(10) * d * mu_err
    print(f'd = ({d/1e6:.2f} + {d_err[0]/1e6:.2f} - {d_err[1]/1e6:.2f}) Mpc')
    
    return d, d_err

d,d_err = distance_modulus_to_parsec(30.033,np.array([0.014,0.015]))

## Fit with emcee

In [ ]:
import emcee
from scipy.optimize import minimize
from inspect import signature
import corner

class MaximumLikelihood:
    '''

    for uncertainties 
    https://erikbern.com/2018/10/08/the-hackers-guide-to-uncertainty-estimates.html
    
    Parameters
    ----------
    func : function
        PDF of the form `func(data,params)`. `func` must accept a
        ndarray for `data` and can have any number of additional
        parameters (at least one).
        
    data : ndarray
        Measured data that are feed into `func`.

    err : ndarray
        Error associated with data.

    prior : function
        Prior probabilities for the parameters of func.

    method : 
        algorithm that is used for the minimization.

    **kwargs
       additional fixed key word arguments that are passed to func.
    '''
    
    def __init__(self,func,data,err=None,prior=None,method='Nelder-Mead',**kwargs):
        
        if len(signature(func).parameters)-len(kwargs)<2:
            raise ValueError(f'`func` must have at least one free argument')
        
        self.func   = func
        self.labels = list(signature(func).parameters.keys())[1:]
        self.data   = data
        self.err    = err
        self.prior  = prior
        self.method = method
        self.kwargs = kwargs

        logger.info(f'searching best parameters for {self.labels} with {len(self.data)} data points')

    def log_likelihood(self,params,data):
        '''calculate the log liklihood of the given parameters
        
        This function takes the previously specified PDF and calculates
        the sum of the logarithmic probabilities. If key word arguments
        were initially passed to the class, they are also passed to the
        function
        '''
        return np.sum(np.log(self.func(data,*params,**self.kwargs)))
    
    def log_prior(self,params):
        '''log of priors'''
        
        lp = self.prior(*params)
        if lp == 0:
            return -np.inf
        else:
            return np.log(lp)

    def log_probability(self,params,data):
        if self.prior:
            return self.log_likelihood(params,data)+self.log_prior(params)
        else:
            return self.log_likelihood(params,data)
        
    def neg_log_probability(self,params,data):
        if self.prior:
            return -self.log_likelihood(params,data)+self.log_prior(params)
        else:
            return -self.log_likelihood(params,data)
        
    def fit(self,guess,nwalkers=10,nsteps=1000):
        '''use scipy minimize to find the best parameters'''
        
        self.soln = minimize(self.neg_log_probability,guess,args=(self.data,),method=self.method)
        if not self.soln.success:
            raise RuntimeError('fit was not successful')
        self.x = self.soln.x

        ndim = len(guess)
        pos = np.random.normal(self.soln.x,0.1,size=(nwalkers,ndim))

        self.sampler = emcee.EnsembleSampler(nwalkers, ndim,self.log_probability,args=(self.data,))
        state = self.sampler.run_mcmc(pos, 100)
        self.sampler.reset()
        self.sampler.run_mcmc(state,nsteps,progress=True)

        flat_samples = self.sampler.get_chain(discard=100,thin=5, flat=True)

        for i in range(ndim):
            mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
            mcmc[1] = self.soln.x[i]
            q = np.diff(mcmc)
            #truth.append(mcmc[1])
            print(f'{self.labels[i]}= {mcmc[1]:.3f}+{q[1]:.3f}-{q[0]:.3f}') 
        
        fig = corner.corner(flat_samples,bins=100, labels=self.labels,truths=self.x)
        
        return self.x

    def __call__(self,guess,**kwargs):
        '''use scipy minimize to find the best parameters'''

        return self.fit(guess,**kwargs)


In [ ]:
MLE = MaximumLikelihood(pnlf,
                        data,
                        mhigh=27)
                        #err=err,
                        #mhigh=galaxy.completeness_limit)

MLE.fit([29],nwalkers=100)    

In [ ]:
data = galaxy.OIII5006_DAP
fig, ax = plt.subplots(figsize=(10,10),subplot_kw={'projection':galaxy.wcs})


norm = simple_norm(data,'linear',clip=False,max_percent=95)

x,y,d = 30,600,50
mask = slice(x,x+d),slice(y,y+d)
ax.imshow(data,norm=norm,cmap=plt.cm.Blues_r)
rect = mpl.patches.Rectangle((x,y),d,d,linewidth=0.8,edgecolor='r',facecolor='none')
ax.add_patch(rect)
plt.show()

In [ ]:
data = galaxy.HA6562
fig, ax = plt.subplots(figsize=(10,10),subplot_kw={'projection':galaxy.wcs})

x,y,d = 200,680,100
mask = slice(x,x+d),slice(y,y+d)

norm = simple_norm(data[mask],'linear',clip=False,max_percent=95)

ax.imshow(data[mask],norm=norm,cmap=plt.cm.Blues_r)
#rect = mpl.patches.Rectangle((x,y),d,d,linewidth=0.8,edgecolor='r',facecolor='none')
#ax.add_patch(rect)
plt.show()

In [ ]:
sigma_clipped_stats(data[mask])

In [ ]:
sigma_clipped_stats(galaxy.HA6562_err[mask])

## HST Data

In [ ]:
file = data_raw / 'NGC628_HST' / 'ngc628_acs_f435w_exp_drc_sci.fits'

with fits.open(file) as hdul:
    #hdul.info()
    data = hdul[0].data
    header = hdul[0].header
    
data[data==0] = np.nan

In [ ]:
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(111,projection=WCS(header))

norm = simple_norm(data,'log',clip=False,min_percent=3,max_percent=99)
ax.imshow(data,norm=norm)
ax.set_xlim([3000,10000])
ax.set_ylim([3000,10000])

filename = basedir / 'reports' / f'NGC628_HST.pdf'
plt.savefig(filename,dpi=1000)

## Compare to other papers

In [ ]:
from pymuse.auxiliary import search_table, Distance
from photutils import CircularAperture

raw = ascii.read(basedir / 'data' / 'external' / 'Herrmann_2008_pn_candidates.txt')

raw['RA'] = 12*' '
raw['DEC'] = 13*' '

for row in raw:
    row['RA'] = f'{row["RAh"]:02d}h{row["RAm"]:02d}m{row["RAs"]:.0f}s'
    row['DEC'] = f'{row["DEd"]:02d}d{row["DEm"]:02d}m{row["DEs"]:0f}s'

raw['SkyCoord'] = SkyCoord(raw['RA'],raw['DEC'])

In [ ]:
catalogcoord = tbl[tbl['type']=='PN']

matchcoord   = search_table(raw,'M74')
matchcoord['x'],matchcoord['y']= matchcoord['SkyCoord'].to_pixel(wcs=galaxy.wcs)

In [ ]:
matchcoord['in_frame'] = False
y_dim,x_dim = galaxy.shape

for row in matchcoord:
    txt,x,y = row['ID'], row['x']+5, row['y']    
    if 0<=int(x)<x_dim and 0<=int(y)<y_dim:
        if not np.isnan(galaxy.PSF[int(y),int(x)]):
            row['in_frame'] = True

data = [d for d in matchcoord['m5007']]
data = matchcoord[matchcoord['in_frame']]['m5007']

In [ ]:
completeness = 26.6

data = matchcoord[matchcoord['in_frame']]['m5007']
data = data[data<completeness]
random.shuffle(data)
fitter = MaximumLikelihood1D(pnlf,
                             np.array(data),
                             mhigh=completeness)


mu,dm_p,dm_m = fitter.plot()

In [ ]:
binsize = 0.25
plot_pnlf(matchcoord['m5007'],29.91,26.5,binsize=binsize,mhigh=28)

In [ ]:
binsize = 0.25
plot_pnlf(matchcoord[matchcoord['in_frame']]['m5007'],29.91,26.5,binsize=binsize,mhigh=28)

In [ ]:
matchcoord=matchcoord[:40]

fig = plt.figure(figsize=(6.974,6.974))
ax1 = fig.add_subplot(111,projection=galaxy.wcs)

norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax1.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Blues_r)

ax1.scatter(matchcoord['x'],matchcoord['y'],marker='o',s=2,c='tab:red')
for row in matchcoord:
    txt,x,y = row['ID'], row['x']+5, row['y']    
    ax1.annotate(txt, (x, y),fontsize=4,color='tab:red')
    
plt.savefig('test.pdf',dpi=600)

In [ ]:
matchcoord = matchcoord[matchcoord['in_frame']][:40]

fig = plt.figure(figsize=(6.974,6.974))
ax1 = fig.add_subplot(111,projection=galaxy.wcs)

norm = simple_norm(galaxy.whitelight,'linear',clip=False,max_percent=95)
ax1.imshow(galaxy.whitelight,norm=norm,cmap=plt.cm.Blues)

positions = np.transpose([matchcoord['x'],matchcoord['y']])
apertures = CircularAperture(positions, r=6)
apertures.plot(color='tab:red',lw=.3, alpha=1,ax=ax1)

positions = np.transpose([catalogcoord['x'],catalogcoord['y']])
apertures = CircularAperture(positions, r=6)
apertures.plot(color='tab:orange',lw=.3, alpha=1,ax=ax1)

for row in matchcoord:
    txt,x,y = row['ID'], row['x']+5, row['y']    
    ax1.annotate(txt, (x, y),fontsize=4,color='tab:red')
    
plt.savefig('test.pdf',dpi=600)

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)

tolerance = '0.5s'
ID, angle, Quantity  = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle.mean().to_string(u.arcsec,decimal=True)}"')